# kartograafr: Query Canvas and AGOL


## _⚠️ WARNING ⚠️_
> Whenever this notebook is updated, before committing changes back to the git repository, **_remove all output_** from cells.  The output may contain PII about students, which should not be made public.

## Import kartograafr Code

> 💡**_Note_**: For some reason, in `main.py` the line `logger = None` needs to be commented out when used in this notebook.  It doesn't affect how the program runs outside of the notebook, though.

In [ ]:
from main import *

# TODO: Move logging init to function, which is called by main()

## Connect to Canvas and ArcGIS Online (AGOL)

### Set AGOL Connection Mode
Set `pro` to `True` below to connect to AGOL using _ArcGIS Pro_, which requires local installation of that application.

In [ ]:
pro = False

### Connect to AGOL

In [ ]:
if (pro):
    from arcgis import GIS
    arcGIS = GIS('pro')

else:
    arcGIS = arcgisUM.getArcGISConnection(config.ArcGIS.SECURITYINFO)

print( 'Login successful!' )
print( '  server: ' + arcGIS.properties.name )
print( '  user: ' + arcGIS.properties.user.username )
print( '  role: ' + arcGIS.properties.user.role )

arcGIS

In [ ]:
canvas = getCanvasInstance()
canvas

## Query Canvas

### Find Valid Outcome Object by ID

In [ ]:
outcomeID = config.Canvas.TARGET_OUTCOME_ID
validOutcome = canvas.getOutcomeObject(outcomeID)

if not validOutcome:
    raise RuntimeError('Outcome ID {} not found'.format(outcomeID))

### Find a Specific Course by ID and Outcome

In [ ]:
courseID = 192894
matchingCourseIDs = getCourseIDsWithOutcome(canvas, [courseID], validOutcome)

if len(matchingCourseIDs) == 0:
    raise RuntimeError('Course {} does not have outcome {}'.format(courseID, outcomeID))

### Find Assignments from Course with Outcome

**_Note_**: The first part of the output is from logging statements.  The second part is only the course's assignments with the desired outcome.

In [ ]:
# TODO: Move the logging somewhere else
matchingCourseAssignments = getCourseAssignmentsWithOutcome(canvas, matchingCourseIDs, validOutcome)

matchingCourseAssignments

### Get Full Data of Matching Courses

In [ ]:
# TODO: Combine with function that only returns matchingCourseIDs?
courseDictionary = getCoursesByID(canvas, matchingCourseIDs)
courseDictionary

### Get Courses' Users by Course IDs

In [ ]:
courseUserDictionary = getCoursesUsersByID(canvas, matchingCourseIDs)
courseUserDictionary

### Get Data for Specific Course and Assignment

Using the data structures returned by the functions above, pick out the full data for the 
specific course (mentioned earlier) and for the first of all the assignments.

In [ ]:
course = courseDictionary[courseID]
assignment = matchingCourseAssignments[0] # Only the first assignment

## Query AGOL

### Construct the AGOL Assignment Group Name

In [ ]:
groupTitle = '%s_%s_%s_%s' % (course.name, course.id, assignment.name, assignment.id)
groupTitle

### Query AGOL for the Group

In [ ]:
group = arcgisUM.lookForExistingArcGISGroup(arcGIS, groupTitle)
group

### Get List of Group Users

**_Note_**: People are added to AGOL with username in the form of:

"_`uniqname`_`_umich`"

To compare with the usernames (AKA `login_id`) from Canvas, remove the "`_umich`" parts.

In [ ]:
groupUsers = arcgisUM.getCurrentArcGISMembers(group, 'groupNameAndID')
groupUsersTrimmed = [re.sub('_\S+$', '', gu) for gu in groupUsers]
groupUsersTrimmed

## Compare Users from AGOL and Canvas

### Make List of Canvas Usernames

In [ ]:
canvasCourseUsers = [user.login_id for user in courseUserDictionary[course.id] if user.login_id is not None]
canvasCourseUsers

## TODO: Complete This Notebook

This notebook was started as a way to debug a specific problem.  At this point, the problem was revealed and nothing more was needed.  At some point, this notebook should be completed to demonstrate the other steps of the kartograafr process.

In addition to debugging a specific problem, this notebook also points out a few things that should be changed in the kartograafr code:

* To be used easily in Jupyter Notebook, it should become more like an API.
* Some functions log info, which is very useful when the program is running in production, but maybe not always desired in notebook output.
* Improve logging initialization, including a way to gracefully disable logging.
* Rather than querying for course IDs in one place, then query again for their full data, why not go for the full data in the first place?  Reduce the number of API calls.